In [1]:
import pandas as pd

file = "20.csv"
df = pd.read_csv(file, encoding='latin1')

df

,COMMUNE_CODE,COMMUNE_NOM,SEXE,AGE00_05,AGE05_10,AGE10_15,AGE15_20,AGE20_25,AGE25_30,AGE30_35,...,AGE55_60,AGE60_65,AGE65_70,AGE70_75,AGE75_80,AGE80_85,AGE85_90,AGE90_95,AGE95_100,AGE100_
0,1101,Beaufort,F,76,86,109,88,84,92,115,...,95,70,66,41,35,19,16,6,0,0
1,1101,Beaufort,M,92,89,81,95,98,86,109,...,119,86,63,31,30,11,14,2,0,0
2,1102,Bech,F,36,24,43,41,48,36,31,...,40,41,35,28,10,16,8,2,0,0
3,1102,Bech,M,36,37,38,44,43,32,38,...,48,50,38,26,13,15,7,1,0,0
4,802,Beckerich,F,74,70,72,73,99,95,106,...,108,103,57,44,39,17,23,7,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,601,Wincrange,M,136,143,131,130,162,160,146,...,190,159,132,86,54,29,17,8,0,0
200,913,Winseler,F,36,31,24,41,42,42,35,...,47,40,41,37,21,14,7,2,0,2
201,913,Winseler,M,32,23,26,47,49,48,37,...,74,51,53,44,24,11,4,3,0,0
202,1209,Wormeldange,F,87,87,75,64,83,83,118,...,123,89,101,61,35,38,19,8,1,0


In [2]:
# === 2. Identifica le colonne di età ===
age_cols = [c for c in df.columns if c.startswith("AGE")]

# === 3. Pulisci eventuali spazi o caratteri anomali e converti in numeri ===
for c in age_cols:
    df[c] = pd.to_numeric(df[c].astype(str)
                          .str.replace("\u00A0", "", regex=False)
                          .str.replace(" ", "", regex=False)
                          .str.replace(",", "."),
                          errors="coerce")

# === 4. Calcola popolazione 65+ e totale ===
cols_65plus = [c for c in age_cols if int(c[3:5]) >= 65]  # età >= 65
df["TOT65PLUS"] = df[cols_65plus].sum(axis=1)
df["TOTALPOP"] = df[age_cols].sum(axis=1)

# === 5. Aggrega per comune (sommando maschi + femmine) ===
agg_df = df.groupby(["COMMUNE_CODE", "COMMUNE_NOM"], as_index=False)[["TOT65PLUS", "TOTALPOP"]].sum()

# === 6. Calcola percentuale ===
agg_df["PERC65PLUS"] = agg_df["TOT65PLUS"] / agg_df["TOTALPOP"] * 100

# === 7. Risultato finale per l’anno 2020 ===
agg_df = agg_df.rename(columns={
    "TOT65PLUS": "TOT65PLUS_2020",
    "PERC65PLUS": "PERC65PLUS_2020"
})

agg_df.head()


,COMMUNE_CODE,COMMUNE_NOM,TOT65PLUS_2020,TOTALPOP,PERC65PLUS_2020
0,1,Luxembourg,15032,124045,12.118183
1,203,Dippach,690,4427,15.586176
2,204,Garnich,306,2204,13.883848
3,206,Kehlen,1154,6226,18.535175
4,207,Koerich,358,2630,13.612167


In [4]:
import pandas as pd
import glob

# === Funzione per leggere e aggregare un singolo file ===
def process_file(path):
    # Estrai l'anno dal nome file (es. "20.csv" → 2020)
    year = "20" + path.split(".")[0][-2:]
    
    # Leggi il file
    df = pd.read_csv(path, encoding='latin1')
    
    # Identifica colonne età
    age_cols = [c for c in df.columns if c.startswith("AGE")]
    
    # Pulisci valori e converti in numerico
    for c in age_cols:
        df[c] = pd.to_numeric(
            df[c].astype(str)
                 .str.replace("\u00A0", "", regex=False)
                 .str.replace(" ", "", regex=False)
                 .str.replace(",", "."),
            errors="coerce"
        )

    # Colonne età 65+
    cols_65plus = [c for c in age_cols if int(c[3:5]) >= 65]
    
    # Totali per riga (maschi/femmine)
    df["TOT65PLUS"] = df[cols_65plus].sum(axis=1)
    df["TOTALPOP"] = df[age_cols].sum(axis=1)

    # Aggrega per comune (somma maschi + femmine)
    agg = df.groupby(["COMMUNE_CODE", "COMMUNE_NOM"], as_index=False)[["TOT65PLUS", "TOTALPOP"]].sum()

    # Percentuale
    agg[f"PERC65PLUS_{year}"] = agg["TOT65PLUS"] / agg["TOTALPOP"] * 100
    agg = agg.rename(columns={"TOT65PLUS": f"TOT65PLUS_{year}"})
    agg = agg.drop(columns=["TOTALPOP"])

    return agg

# === Applica a tutti i file 20–25 ===
files = sorted(glob.glob("2[0-5].csv"))  # trova tutti i file 20.csv–25.csv
dataframes = [process_file(f) for f in files]

# === Unisci progressivamente tutti i dataset per comune ===
from functools import reduce
df_final = reduce(lambda left, right: pd.merge(left, right, on=["COMMUNE_CODE", "COMMUNE_NOM"], how="outer"), dataframes)

# === Ordina per codice comune ===
df_final = df_final.sort_values("COMMUNE_CODE").reset_index(drop=True)

# === Salva il dataset finale ===
df_final.to_csv("Population_65plus_by_commune.csv", index=False)

print("✅ Dataset finale creato: Population_65plus_by_commune.csv")
print(df_final.head())


✅ Dataset finale creato: Population_65plus_by_commune.csv
   COMMUNE_CODE COMMUNE_NOM  TOT65PLUS_2020  PERC65PLUS_2020  TOT65PLUS_2021  \
0             1  Luxembourg         15032.0        12.118183         15274.0   
1           203     Dippach           690.0        15.586176           717.0   
2           204     Garnich           306.0        13.883848           320.0   
3           206      Kehlen          1154.0        18.535175          1189.0   
4           207     Koerich           358.0        13.612167           377.0   

   PERC65PLUS_2021  TOT65PLUS_2022  PERC65PLUS_2022  TOT65PLUS_2023  \
0        11.972377         15699.0        11.901296         16043.0   
1        15.990187           757.0        16.542832           786.0   
2        14.103129           338.0        14.844093           356.0   
3        19.251943          1244.0        19.683544          1283.0   
4        14.269493           391.0        14.562384           395.0   

   PERC65PLUS_2023  TOT65PLUS_2024

In [17]:
df_final

,COMMUNE_CODE,COMMUNE_NOM,TOT65PLUS_2020,PERC65PLUS_2020,TOT65PLUS_2021,PERC65PLUS_2021,TOT65PLUS_2022,PERC65PLUS_2022,TOT65PLUS_2023,PERC65PLUS_2023,TOT65PLUS_2024,PERC65PLUS_2024,TOT65PLUS_2025,PERC65PLUS_2025
0,1,Luxembourg,15032.0,12.118183,15274.0,11.972377,15699.0,11.901296,16043.0,11.942443,16273.0,11.947169,16546.0,12.074640
1,203,Dippach,690.0,15.586176,717.0,15.990187,757.0,16.542832,786.0,17.131648,824.0,17.655882,844.0,17.980401
2,204,Garnich,306.0,13.883848,320.0,14.103129,338.0,14.844093,356.0,15.471534,366.0,15.837300,376.0,15.738803
3,206,Kehlen,1154.0,18.535175,1189.0,19.251943,1244.0,19.683544,1283.0,19.186481,1353.0,18.928372,1390.0,18.908992
4,207,Koerich,358.0,13.612167,377.0,14.269493,391.0,14.562384,395.0,14.400292,427.0,15.343155,441.0,15.677213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,1306,Remich,847.0,22.526596,863.0,22.734457,900.0,22.545090,926.0,22.629521,931.0,22.439142,959.0,22.986577
100,1307,Stadtbredimus,258.0,13.149847,271.0,13.770325,271.0,13.756345,290.0,14.580191,314.0,15.629667,322.0,15.932707
101,1308,Waldbredimus,190.0,15.067407,191.0,15.098814,197.0,14.935557,3.0,30.000000,3.0,30.000000,3.0,30.000000
102,1310,Schengen,776.0,15.743559,822.0,16.193853,852.0,16.441528,878.0,16.966184,911.0,17.468840,937.0,17.847619


In [18]:
df = pd.read_csv("Population_65plus_by_commune.csv")

df

,COMMUNE_CODE,COMMUNE_NOM,TOT65PLUS_2020,PERC65PLUS_2020,TOT65PLUS_2021,PERC65PLUS_2021,TOT65PLUS_2022,PERC65PLUS_2022,TOT65PLUS_2023,PERC65PLUS_2023,TOT65PLUS_2024,PERC65PLUS_2024,TOT65PLUS_2025,PERC65PLUS_2025
0,1,Luxembourg,15032.0,12.118183,15274.0,11.972377,15699.0,11.901296,16043.0,11.942443,16273.0,11.947169,16546.0,12.074640
1,203,Dippach,690.0,15.586176,717.0,15.990187,757.0,16.542832,786.0,17.131648,824.0,17.655882,844.0,17.980401
2,204,Garnich,306.0,13.883848,320.0,14.103129,338.0,14.844093,356.0,15.471534,366.0,15.837300,376.0,15.738803
3,206,Kehlen,1154.0,18.535175,1189.0,19.251943,1244.0,19.683544,1283.0,19.186481,1353.0,18.928372,1390.0,18.908992
4,207,Koerich,358.0,13.612167,377.0,14.269493,391.0,14.562384,395.0,14.400292,427.0,15.343155,441.0,15.677213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,1306,Remich,847.0,22.526596,863.0,22.734457,900.0,22.545090,926.0,22.629521,931.0,22.439142,959.0,22.986577
100,1307,Stadtbredimus,258.0,13.149847,271.0,13.770325,271.0,13.756345,290.0,14.580191,314.0,15.629667,322.0,15.932707
101,1308,Waldbredimus,190.0,15.067407,191.0,15.098814,197.0,14.935557,3.0,30.000000,3.0,30.000000,3.0,30.000000
102,1310,Schengen,776.0,15.743559,822.0,16.193853,852.0,16.441528,878.0,16.966184,911.0,17.468840,937.0,17.847619


In [20]:
# 2. Define mapping of incorrect → correct names
name_fix = {
    "Esch-sur-SÃ»re": "Esch-sur-Sûre",
    "PrÃ©izerdaul": "Préizerdaul",
    "Erpeldange-sur-SÃ»re": "Erpeldange-sur-Sûre",
    "PÃ©tange": "Pétange",
    "KÃ¤erjeng": "Käerjeng",
    "VallÃ©e de l'Ernz": "Vallée de l'Ernz",
    "Lac de la Haute-SÃ»re": "Lac de la Haute-Sûre"
}

# 3. Apply corrections to the commune name column
df["COMMUNE_NOM"] = df["COMMUNE_NOM"].replace(name_fix)

# 4. Save the corrected file
df.to_csv("data_fixed.csv", index=False, encoding="utf-8-sig")

In [23]:
# Comuni interessati da fusione
merge_map = {
    "Bous-Waldbredimus": ["Bous", "Waldbredimus"],
    "Groussbus-Wal": ["Grosbous", "Wahl"]
}

# Mostra tutte le righe corrispondenti
affected = df[df["COMMUNE_NOM"].isin(
    sum(merge_map.values(), []) + list(merge_map.keys())
)]
print(affected.sort_values("COMMUNE_NOM"))


     COMMUNE_CODE        COMMUNE_NOM  TOT65PLUS_2020  PERC65PLUS_2020  \
95           1300               Bous           206.0        12.225519   
103          1311  Bous-Waldbredimus             NaN              NaN   
63            807           Grosbous           141.0        12.737127   
69            814      Groussbus-Wal             NaN              NaN   
68            813               Wahl           126.0        12.034384   
101          1308       Waldbredimus           190.0        15.067407   

     TOT65PLUS_2021  PERC65PLUS_2021  TOT65PLUS_2022  PERC65PLUS_2022  \
95            219.0        12.822014           222.0        12.802768   
103             NaN              NaN             NaN              NaN   
63            154.0        13.628319           163.0        14.579606   
69              NaN              NaN             NaN              NaN   
68            128.0        12.260536           130.0        11.970534   
101           191.0        15.098814           197

In [24]:
import pandas as pd
import numpy as np

# === Mappa corretta delle fusioni ===
merge_map = {
    "Bous-Waldbredimus": ["Bous", "Waldbredimus"],
    "Groussbus-Wal": ["Grosbous", "Wahl"]
}

# === Copia del dataset originale ===
df_merged = df.copy()

for new_name, old_names in merge_map.items():
    # Filtra tutte le righe interessate (nuovo + vecchi)
    subset = df_merged[df_merged["COMMUNE_NOM"].isin([new_name] + old_names)]
    if subset.empty:
        print(f"⚠️ Nessuna riga trovata per {new_name}")
        continue

    # Colonne da sommare e medie
    abs_cols = [c for c in df_merged.columns if c.startswith("TOT65PLUS_")]
    perc_cols = [c for c in df_merged.columns if c.startswith("PERC65PLUS_")]

    # Somma assoluti, media percentuali
    sum_vals = subset[abs_cols].sum(numeric_only=True)
    mean_vals = subset[perc_cols].mean(numeric_only=True)

    # Costruisci nuova riga aggregata
    new_row = {
        "COMMUNE_CODE": subset["COMMUNE_CODE"].iloc[0],  # mantiene il primo codice
        "COMMUNE_NOM": new_name,
        **sum_vals.to_dict(),
        **mean_vals.to_dict()
    }

    # Rimuovi le righe vecchie e aggiungi quella nuova
    df_merged = df_merged[~df_merged["COMMUNE_NOM"].isin([new_name] + old_names)]
    df_merged = pd.concat([df_merged, pd.DataFrame([new_row])], ignore_index=True)

# === Riordina per nome e resetta indice ===
df_merged = df_merged.sort_values("COMMUNE_NOM").reset_index(drop=True)

# === Stampa risultato per verifica ===
print("✅ Comuni fusi correttamente:\n")
print(df_merged[df_merged["COMMUNE_NOM"].isin(merge_map.keys())])


✅ Comuni fusi correttamente:

    COMMUNE_CODE        COMMUNE_NOM  TOT65PLUS_2020  PERC65PLUS_2020  \
12          1300  Bous-Waldbredimus           396.0        13.646463   
35           807      Groussbus-Wal           267.0        12.385756   

    TOT65PLUS_2021  PERC65PLUS_2021  TOT65PLUS_2022  PERC65PLUS_2022  \
12           410.0        13.960414           419.0        13.869163   
35           282.0        12.944427           293.0        13.275070   

    TOT65PLUS_2023  PERC65PLUS_2023  TOT65PLUS_2024  PERC65PLUS_2024  \
12           443.0         14.70387           462.0        14.696133   
35           309.0         13.54669           319.0        13.562925   

    TOT65PLUS_2025  PERC65PLUS_2025  
12           477.0        14.731956  
35           322.0        13.427857  


In [26]:
import json

# === Crea struttura nested per anno ===
records = []
for _, row in df_merged.iterrows():
    commune_data = {
        "COMMUNE_CODE": row["COMMUNE_CODE"],
        "COMMUNE_NOM": row["COMMUNE_NOM"],
        "data": {}
    }
    
    # Cicla sugli anni
    years = sorted(set(c.split("_")[-1] for c in df_merged.columns if c.startswith("TOT65PLUS_")))
    for year in years:
        commune_data["data"][year] = {
            "TOT65PLUS": float(row[f"TOT65PLUS_{year}"]) if pd.notnull(row[f"TOT65PLUS_{year}"]) else None,
            "PERC65PLUS": float(row[f"PERC65PLUS_{year}"]) if pd.notnull(row[f"PERC65PLUS_{year}"]) else None
        }
    
    records.append(commune_data)

# === Salva in JSON ===
with open("Population_65plus_merged.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print("💾 File JSON salvato come 'Population_65plus_merged.json'")


💾 File JSON salvato come 'Population_65plus_merged.json'


In [27]:
df_merged.to_csv("data_total.csv", index=False)